## This program creates Neighborhood Delination across different time period


In [67]:
#!/usr/bin/env python
# coding: utf-8
import json, math, copy, sys
import pandas as pd
import shapely.wkt
import shapely.geometry
from datetime import datetime
from datetime import timedelta
from pathlib import Path
import urllib.parse
import webbrowser
import os
import pprint
from sklearn.preprocessing import minmax_scale
import numpy as np
from scipy import stats
from notebook import notebookapp
from IPython.core.display import display, HTML
import geopandas as gpd

### Set your input parameters and method to run clustering
select your MSA or county id here: http://su-gis.iptime.org/LNE/pick_POI.html

In [68]:
param = {
    'title': "Neighborhood, Cook County (zipcode level)",
    'subject': "NEIGHBORHOOD",
    'filename_suffix': "Cook_2018_from_ACS_zipcode_from_file", 
    'Layers': [1980,2000,2010],
    'inputCSV': "attributes/Input_attributes.csv",   
    'shapefile': "shp/Cook_County_Tract.shp", 
    'Maps_of_neighborhood': True,                #choropleth map: Maps representing clustering result		
    'Temporal_change_in_neighborhoods': False,    #stacked chart: Temporal Change in Neighborhoods over years		
    'Parallel_Categories_Diagram_in_neighborhoods': False,
    'Chord_Diagram_in_neighborhoods': True
}

## Write index.html

In [69]:
#Create a new folder where GEO_CONFIG.js GEO_JSON.js VARIABLES.js will be saved
oDir = 'QUAL_' + param['filename_suffix']
path = Path(oDir + '/data')
path.mkdir(parents=True, exist_ok=True)

contents = []
#open Neighborhood_Analysis_Mapper.html (the excutable file for the visualization)
ifile = open("template/Qualitative_Analysis_Mapper.html", "r", encoding="utf-8")
contents = ifile.read()

#Replace variables based on the user's selection in each of four files below.
contents = contents.replace("Neighborhood Analysis Mapper", param['title'])
contents = contents.replace("data/CONFIG.js", "data/CONFIG_"+param['filename_suffix']+".js")
contents = contents.replace("data/GEO_JSON.js", "data/GEO_JSON_"+param['filename_suffix']+".js")
contents = contents.replace("data/VARIABLES.js", "data/VARIABLES_"+param['filename_suffix']+".js")

#write new outfiles: GEO_CONFIG.js GEO_JSON.js VARIABLES.js
ofile = open(oDir+"/index.html", "w", encoding="utf-8")
ofile.write(contents)
ofile.close()

print (contents)

<!DOCTYPE html>
<html>
<head>
	<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>
	<title>Neighborhood, Cook County (zipcode level)</title>

	<script src="data/CONFIG_Cook_2018_from_ACS_zipcode_from_file.js"></script>
	<script src="data/GEO_JSON_Cook_2018_from_ACS_zipcode_from_file.js"></script>
	<script src="data/VARIABLES_Cook_2018_from_ACS_zipcode_from_file.js"></script>

	<script src="../template/Neighborhood_Analysis_Mapper/lib/jQuery/jquery-3.3.1.js"></script>
	<script src="../template/Neighborhood_Analysis_Mapper/lib/d3/d3.min.js"></script>
	<!--script src="lib/d3-chord-diagrams-master/lib/d3.js"></script-->
	<!--script src="lib/d3-chord-diagrams-master/lib/underscore.js"></script>
	<script src="lib/d3-chord-diagrams-master/js/mapper.js"></script-->
	<script src="../template/Neighborhood_Analysis_Mapper/lib/moment/moment.min.js"></script>
	<script src="../template/Neighborhood_Analysis_Mapper/lib/geostats/geostats.min.js"></script>
	<script src="../template/Nei

## Write GEO_CONFIG_XXX.js

In [70]:
# read ACM_GEO_CONFIG.js
ifile = open("template/QUAL_CONFIG.js", "r", encoding="utf-8")
contents = ifile.read()

SubjectName = "";
Maps_of_neighborhood = True;               
Temporal_change_in_neighborhoods = True;
Parallel_Categories_Diagram_in_neighborhoods = True;
Chord_Diagram_in_neighborhoods = True;

if ('subject' in param): SubjectName =  param['subject']
if ('Maps_of_neighborhood' in param): Maps_of_neighborhood =  param['Maps_of_neighborhood']
if ('Temporal_change_in_neighborhoods' in param): Temporal_change_in_neighborhoods =  param['Temporal_change_in_neighborhoods']
if ('Parallel_Categories_Diagram_in_neighborhoods' in param): Parallel_Categories_Diagram_in_neighborhoods =  param['Parallel_Categories_Diagram_in_neighborhoods']
if ('Chord_Diagram_in_neighborhoods' in param): Chord_Diagram_in_neighborhoods =  param['Chord_Diagram_in_neighborhoods']

InitialLayers = []
if (len(param['Layers']) <= 1): InitialLayers = []
for i, year in enumerate(param['Layers']):
	InitialLayers.append(str(year))
InitialLayers

['1980', '2000', '2010']

In [71]:
NumOfMaps = len(InitialLayers)
# Automatically set Map_width, Map_height. 
Map_width = "300px"
Map_height = "300px"
if (NumOfMaps <= 6):
	Map_width = "300px"
	Map_height = "300px"	
if (NumOfMaps <= 5):
	Map_width = "350px"
	Map_height = "350px"
if (NumOfMaps <= 4):
	Map_width = "400px"
	Map_height = "400px"
if (NumOfMaps <= 3):
	Map_width = "400px"
	Map_height = "400px"
if (NumOfMaps <= 2):
	Map_width = "450px"
	Map_height = "450px"
if (NumOfMaps ==	1):
	Map_width = "800px"
	Map_height = "800px"
    
# replace newly computed "NumOfMaps", "InitialLayers", "Map_width", "Map_height" in CONFIG.js. See the example replacement below
InitialLayers = "var InitialLayers = " + json.dumps(InitialLayers) + ";"
SubjectName = 'var SubjectName = "' + SubjectName + '";'
Maps_of_neighborhood = "var Maps_of_neighborhood = " + json.dumps(Maps_of_neighborhood)+ ";"
Temporal_change_in_neighborhoods = "var Temporal_change_in_neighborhoods = " + json.dumps(Temporal_change_in_neighborhoods)+ ";"
Parallel_Categories_Diagram_in_neighborhoods = "var Parallel_Categories_Diagram_in_neighborhoods = " + json.dumps(Parallel_Categories_Diagram_in_neighborhoods)+ ";"
Chord_Diagram_in_neighborhoods = "var Chord_Diagram_in_neighborhoods = " + json.dumps(Chord_Diagram_in_neighborhoods)+ ";"
Map_width = 'var Map_width  = "' + Map_width + '";'
Map_height = 'var Map_height = "' + Map_height + '";'

contents = contents.replace("var InitialLayers = [];", InitialLayers)
contents = contents.replace('var SubjectName = "";', SubjectName)
contents = contents.replace("var Maps_of_neighborhood = true;", Maps_of_neighborhood)
contents = contents.replace("var Temporal_change_in_neighborhoods = true;", Temporal_change_in_neighborhoods)
contents = contents.replace("var Parallel_Categories_Diagram_in_neighborhoods = true;", Parallel_Categories_Diagram_in_neighborhoods)
contents = contents.replace("var Chord_Diagram_in_neighborhoods = true;", Chord_Diagram_in_neighborhoods)
contents = contents.replace('var Map_width  = "400px";', Map_width)
contents = contents.replace('var Map_height = "400px";', Map_height)

#Write output including the replacement above
filename_GEO_CONFIG = "QUAL_" + param['filename_suffix'] + "/data/CONFIG_"+param['filename_suffix']+".js"
ofile = open(filename_GEO_CONFIG, 'w', encoding="utf-8")
ofile.write(contents)
ofile.close()

print (contents)

// Define the number of maps and some configuration parameters that you want to visualize.
var SubjectName = "NEIGHBORHOOD";
var InitialLayers = ["1980", "2000", "2010"];

/* Map Extent and Zoom level will be automatically adjusted when you do not define map center and zoom level */
//var Initial_map_center = [34.0522, -117.9];  
//var Initial_map_zoom_level = 8;   

var Maps_of_neighborhood = true;							//choropleth map: Maps representing clustering result  
var Temporal_change_in_neighborhoods = false;				//stacked chart: Temporal Change in Neighborhoods over years
var Parallel_Categories_Diagram_in_neighborhoods = false;	//parallel categories diagram
var Chord_Diagram_in_neighborhoods = true;					//chord diagram
  

var Num_Of_Decimal_Places = 2;                             // default = 2

var Map_width  = "400px";                                  // min 350px
var Map_height = "400px";                                  // min 300px


## Write GEO_JSON_XXX.js ===============================================

In [72]:
# read shape file to df_shape
df_shapes = gpd.read_file(param['shapefile'])
df_shapes

,GEOID10,geometry
0,17031010300,"POLYGON ((-87.67133 42.01937, -87.67121 42.019..."
1,17031010400,"POLYGON ((-87.66345 42.01283, -87.66321 42.012..."
2,17031010600,"POLYGON ((-87.67059 42.00537, -87.67046 42.005..."
3,17031020100,"POLYGON ((-87.69024 42.01265, -87.69024 42.012..."
4,17031280800,"POLYGON ((-87.69151 41.88111, -87.69147 41.881..."
...,...,...
1313,17031823901,"POLYGON ((-87.85680 41.67987, -87.85676 41.680..."
1314,17031826800,"POLYGON ((-87.67975 41.64819, -87.67910 41.648..."
1315,17031710300,"POLYGON ((-87.66361 41.75768, -87.66330 41.757..."
1316,17031828506,"POLYGON ((-87.55864 41.54282, -87.55850 41.542..."


In [73]:
df_shapes = df_shapes.rename(columns={'GEOID10': 'geoid'})
df_shapes = df_shapes.astype(str)
geoid = df_shapes.columns[0]
df_shapes = df_shapes[pd.notnull(df_shapes['geometry'])]

# open GEO_JSON.js write heading for geojson format
filename_GEO_JSON = "QUAL_" + param['filename_suffix'] + "/data/GEO_JSON_"+param['filename_suffix']+".js"
ofile = open(filename_GEO_JSON, 'w')
ofile.write('var GEO_JSON =\n')
ofile.write('{"type":"FeatureCollection", "features": [\n')

#Convert geometry in GEOJSONP to geojson format
wCount = 0
for shape in df_shapes.itertuples():
	feature = {"type":"Feature"}
	if (type(shape.geometry) is float):								# check is NaN?
		#print(tract.geometry)
		continue
	#print(tract.geometry)
	aShape = shapely.wkt.loads(shape.geometry)
	feature["geometry"] = shapely.geometry.mapping(aShape)
	#feature["properties"] = {geoid: tract.__getattribute__(geoid), "tractID": tract.__getattribute__(geoid)}
	feature["properties"] = {geoid: shape.__getattribute__(geoid)}
	wCount += 1
	ofile.write(json.dumps(feature)+',\n')
#print("GEO_JSON.js write count:", wCount)
# complete the geojosn format by adding parenthesis at the end.	
ofile.write(']}\n')
ofile.close()

## Write VARIABLES.js ====================================================

In [74]:
#if ('Sequence' not in param or not param['Sequence']): df_pivot.drop(columns=['Sequence'], inplace=True)
df_pivot = pd.read_csv(param["inputCSV"])    
df_pivot.set_index(geoid, inplace=True)
df_pivot

,1980,1990,2000,2010
geoid,,,,
17031010100,2,3,0,2
17031010201,2,2,2,2
17031010202,2,2,2,2
17031010300,2,2,2,2
17031010400,5,5,2,5
...,...,...,...,...
17031843500,3,3,3,4
17031843600,0,0,0,0
17031843700,2,2,2,5


In [75]:
# write df_wide to GEO_VARIABLES.js
filename_GEO_VARIABLES = "QUAL_" + param['filename_suffix'] + "/data/VARIABLES_"+param['filename_suffix']+".js"
ofile = open(filename_GEO_VARIABLES, 'w')
ofile.write('var GEO_VARIABLES =\n')
ofile.write('[\n')

heading = [geoid]
heading.extend(list(map(str, df_pivot.columns.tolist())))
ofile.write('  '+json.dumps(heading)+',\n')
wCount = 0
for i, row in df_pivot.reset_index().iterrows():
	aLine = row.tolist()
	for j, col in enumerate(aLine[2:], 2):
		try:
			aLine[j] = int(col)                                  # convert float to int
		except ValueError:
			aLine[j] = -9999                                     # if Nan, set -9999
	wCount += 1 
	ofile.write('  '+json.dumps(aLine)+',\n')
#print("GEO_VARIABLES.js write count:", wCount)
ofile.write(']\n')
ofile.close()

#Print what's inside GEO_VARIABLES.js
with open(filename_GEO_VARIABLES, 'r') as f2:
    data = f2.read()
    print(data)

var GEO_VARIABLES =
[
  ["geoid", "1980", "1990", "2000", "2010"],
  [17031010100, 2, 3, 0, 2],
  [17031010201, 2, 2, 2, 2],
  [17031010202, 2, 2, 2, 2],
  [17031010300, 2, 2, 2, 2],
  [17031010400, 5, 5, 2, 5],
  [17031010501, 2, 2, 2, 2],
  [17031010502, 2, 2, 2, 2],
  [17031010503, 2, 2, 2, 5],
  [17031010600, 2, 2, 2, 2],
  [17031010701, 2, 2, 2, 2],
  [17031010702, 2, 2, 2, 3],
  [17031020100, 1, 2, 2, 2],
  [17031020200, 1, 1, 1, 2],
  [17031020301, 1, 1, 1, 1],
  [17031020302, 1, 1, 1, 2],
  [17031020400, 2, 2, 2, 2],
  [17031020500, 2, 2, 2, 2],
  [17031020601, 2, 2, 2, 2],
  [17031020602, 2, 2, 2, 2],
  [17031020701, 2, 2, 2, 1],
  [17031020702, 2, 2, 2, 2],
  [17031020801, 2, 2, 2, 2],
  [17031020802, 2, 2, 2, 2],
  [17031020901, 2, 2, 2, 2],
  [17031020902, 2, 2, 2, 2],
  [17031030101, 5, 2, 2, 2],
  [17031030102, 5, 2, 2, 2],
  [17031030103, 5, 2, 2, 2],
  [17031030104, 5, 2, 2, 2],
  [17031030200, 2, 2, 5, 5],
  [17031030300, 2, 2, 2, 2],
  [17031030400, 2, 2, 2, 2],
  [17

### Creating URL to view the visualizatino result

In [76]:
servers = list(notebookapp.list_running_servers())
servers1 = 'https://cybergisx.cigi.illinois.edu'+servers[0]["base_url"]+ 'view'
servers2 = 'https://cybergisx.cigi.illinois.edu'+servers[0]["base_url"]+ 'edit'      
cwd = os.getcwd()
prefix_cwd = "/home/jovyan/work"
cwd = cwd.replace(prefix_cwd, "")
local_dir1 = servers1 + cwd
local_dir2 = servers2 + cwd    
#print(local_dir)
fname =urllib.parse.quote('index.html')
template_dir = os.path.join(local_dir1, 'QUAL_' + param['filename_suffix'])
#url = 'file:' + os.path.join(template_dir, fname)
url = os.path.join(template_dir, fname)    
webbrowser.open(url)
print('To see visualization of your analysis, click the URL below:')
print(url)    
print('Advanced options are available in ')  
print(local_dir2 + '/'+ 'QUAL_' + param['filename_suffix']+'/data/GEO_CONFIG_' + param['filename_suffix']+'.js')

To see visualization of your analysis, click the URL below:
https://cybergisx.cigi.illinois.edu/user/suhanmappingideas/view/geosnap-viz/Qualitative_Geographic_Data_Visualization/QUAL_Cook_2018_from_ACS_zipcode_from_file/index.html
Advanced options are available in 
https://cybergisx.cigi.illinois.edu/user/suhanmappingideas/edit/geosnap-viz/Qualitative_Geographic_Data_Visualization/QUAL_Cook_2018_from_ACS_zipcode_from_file/data/GEO_CONFIG_Cook_2018_from_ACS_zipcode_from_file.js
